# Quora Duplicate Questions

A set of Quora [questions](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) to determine whether pairs of question texts actually correspond to semantically equivalent queries. More than 400,000 lines of potential questions duplicate question pairs. <br>

Objective is problem of identifying duplicate questions. <br>

In [ ]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2021-11-29 15:34:24--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.1.2, 151.101.65.2, 151.101.129.2, ...
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.1.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M   178MB/s    in 0.3s    

2021-11-29 15:34:25 (178 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]



In [ ]:
csv_file = 'quora_duplicate_questions.tsv'

In [ ]:
# csv file can be easily manipulated using pandas
import pandas as pd

In [ ]:
csv_df = pd.read_csv('quora_duplicate_questions.tsv', sep="\t")
csv_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [ ]:
# Extract duplicate rows or similar rows 
quora_dup_question_df = csv_df[csv_df['is_duplicate'] == 1]
quora_dup_question_df

,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1
...,...,...,...,...,...,...
404280,404280,537922,537923,What are some outfit ideas to wear to a frat p...,What are some outfit ideas wear to a frat them...,1
404281,404281,99131,81495,Why is Manaphy childish in Pokémon Ranger and ...,Why is Manaphy annoying in Pokemon ranger and ...,1
404282,404282,1931,16773,How does a long distance relationship work?,How are long distance relationships maintained?,1
404284,404284,537926,537927,What does Jainism say about homosexuality?,What does Jainism say about Gays and Homosexua...,1


In [ ]:
quora_dup_question_df["question1"]

5         Astrology: I am a Capricorn Sun Cap moon and c...
7                            How can I be a good geologist?
11              How do I read and find my YouTube comments?
12                     What can make Physics easy to learn?
13              What was your first sexual experience like?
                                ...                        
404280    What are some outfit ideas to wear to a frat p...
404281    Why is Manaphy childish in Pokémon Ranger and ...
404282          How does a long distance relationship work?
404284           What does Jainism say about homosexuality?
404286            Do you believe there is life after death?
Name: question1, Length: 149263, dtype: object

In [ ]:
quora_dup_question_df["question2"].str.len().max

<bound method Series.max of 5         90
7         41
11        38
12        39
13        38
          ..
404280    55
404281    68
404282    47
404284    51
404286    42
Name: question2, Length: 149263, dtype: int64>

In [ ]:
# Save quora duplicate question as csv file.
quora_dup_question_df.to_csv('quora_duplicate_questions.csv')

In [ ]:
input_file1 = '/content/quora_duplicate_questions.csv'
lines = open(input_file1, encoding='ISO-8859-1').read().strip().split('\n')
for line in lines[0:2]:
  print(line.split(',')[4:6])
    

['question1', 'question2']
['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?', '"I\'m a triple Capricorn (Sun']


In [ ]:
class Lang:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: "SOS", 1 : "EOS"}
    self.n_words = 2

  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.n_words
      self.index2word[self.n_words] = word
      self.word2count[word] = 1
      self.n_words += 1
    else:
      self.word2count[word] += 1

# Turn a Unicode string to Plain ASCII. 
def unicodeToAscii(s):
  return ''.join(
      c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn'
  )

#lowercase, trim, and remove non-letter characters
def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  s = re.sub(r"([.!?])", r" \1",s)
  s = re.sub(r"[^a-zA-Z.!?]+", r" ",s)       
  return s

def read_input_data(src, dest, input_file_path, reverse=False):
  input_data_pairs = []

  # utf-8 format was causing reading error, so using ISO-8859-1 format.
  lines = open(input_file_path, encoding='ISO-8859-1').read().strip().split('\n')
    
  # Split every line into pairs and normalize
  # Column1: Question
  # Column2: Answer
  # Skip header from reading or 1st line.
  pairs = [[normalizeString(s) for s in line.split(sep=',')[4:6] ] for line in lines[1:]]
  input_data_pairs.extend(pairs)

  input_data = Lang(src)
  output_data = Lang(dest)

  return input_data, output_data, input_data_pairs

def readLangs(lang1, lang2, input_file_path, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(input_file_path, encoding='utf-8').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines[1:]]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
  print(p)
  return len(p[0].split(' ')) < MAX_LENGTH and \
    len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

def prepare_data(question1, question2, input_file_path, reverse=False):
  input_data, output_data, pairs = read_input_data(question1, question2, input_file_path, reverse)

  print("Read %s sentence pairs" %len(pairs))
  pairs = filterPairs(pairs)
  
  print("Trimmed to %s sententce pairs" % len(pairs))
  
  print("Counting words...")
  for pair in pairs:
    input_data.addSentence(pair[0])
    output_data.addSentence(pair[1])
  print("counted words:")
  print(input_data.name, input_data.n_words)
  print(output_data.name, output_data.n_words)
  return input_data, output_data, pairs

In [ ]:
input_file_path = '/content/quora_duplicate_questions.csv'
question1 = 'Question1'
question2 = 'Question2'

EOS_Token = 1
SOS_Token = 0


input_data, output_data, pairs = prepare_data(question1, question2, input_file_path, False)
print(random.choice(pairs))

Streaming output truncated to the last 5000 lines.
['do i have ptsd or am i just crazy ?', ' how can i tell if i have ptsd']
['what are the best tools for social media monitoring and why ?', 'what is the best social media monitoring tool and why ?']
['what are the best motivational books for joining the indian army ?', 'what are some interesting books about indian army ?']
['what is the best way to remove permanent marker off a dry erase board ?', 'how can you remove permanent marker stains from dry erase board ?']
['do foreign investors need to be accredited in the us if they invest in delaware registered startups based in other countries ?', 'do foreign investors need to be accredited investors ?']
['what will be the impact of the step taken to ban the rupee note on indian economy ?', 'what will be the effects after banning on and rupee notes ?']
['what is boot process ?', 'what is the booting process ?']
['how would i increase my height aftr ?', 'how do i increase my height after ?'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['can i work in canada on business visa ?', ' i have a valid canada business visa b ']
[' what were the major effects of the cambodia earthquake', 'and how do these effects compare to the tohoku earthquake in ? ']
['does purina make good dog food ?', 'is purina good for dogs ?']
['is it worth going to a university ?', 'is it worth going to university ? what are universities for ?']
['what is the best language learning program ?', 'what is the best program for learning a foreign language ?']
['how does disc brake works in a bike ?', 'how does the disc brake in a motorcycle work ?']
['what is one thing you plan or want to change in your life ?', ' if you were to change one decision in your life']
['how can i get back my instagram deleted dms ?', 'how can i view deleted instagram text messages ?']
['is it possible to marry my mother s sisters husband s brothers daughter ?', 'i want to marry my mother s sister s husband s elder brother s d

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['what if india and pakistan went on war today ?', 'what would happen if war were declared between india and pakistan ?']
['what is the dumbest question you ve seen on quora ?', 'what is the dumbest question you ve seen asked on quora ?']
['what are tracer rounds ? are they legal for the public to possess in the state of connecticut ?', 'what are tracer rounds ? are they legal for the public to possess in the state of delaware ?']
['what are some good restaurants in nagpur ?', 'which is the best restaurent in nagpur ?']
['what are the new features of iphone ?', 'what are some new features in iphone ?']
['how do i speak colombian spanish ?', 'how do you learn colombian spanish ?']
['what piece of advice would you give to a thee for the month of october ? you can share your school days and experiences as well .', 'what piece of advice would you give to a thee in the month of october ? you can share your school days and experiences as wel

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['is donald trump a good choice for being the president of the us ?', 'would donald trump be a good president ?']
['can i get pregnant on my period ?', 'can you get pregnant having sex during your period ?']
['how would india look today if it had not been partitioned ?', 'how would india be today if partition did not happen ?']
['what is the example of presence of mind ?', 'what are some great examples of presence of mind ?']
['why does quora always mark my questions as needing improvement ?', ' why does quora always flag my questions that are well formed']
['which university is considered as the best medical university yin ukraine ?', 'which university is considered as the best medical university in ukraine ?']
['why is china investing in pakistan ?', 'why has china become so obsessed with investing in pakistan ?']
['how do i stop my bad habit of procrastination ?', 'how do you avoid procrastination ?']
['what are the pros and cons of

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
['how are people able to hide their mutual friends on facebook ? what is the way ?', 'how can i hide my mutual friends list on facebook ?']
['what are the best business ideas in ?', 'what are the best business ideas for ?']
['how do i become more confident in front of girls ?', 'i m and i want to become more confident when talking to woman . how can i do this ?']
['what is best business to start in india ?', 'what are the best businesses to start in india ?']
['what are examples of successful one person startups ?', 'what are some examples of very successful one person startup ?']
['which are the modes in which hadoop can run ?', 'what are the modes hadoop can run in ?']
['how does a scrap metal merchant make money from scrap ?', 'how do scrap yards make money ?']
['why does your head hurt after you cry ?', 'why my head hurt when i cry ?']
['can i fly my drone in india ?', 'can i own and fly a drone in india ?']
[' what english dubbed 

In [ ]:
pairs[0:5]

[['what can make physics easy to learn ?',
  'how can you make physics easy to learn ?'],
 ['what was your first sexual experience like ?',
  'what was your first sexual experience ?'],
 ['what does manipulation mean ?', 'what does manipulation means ?'],
 ['why do rockets look white ?',
  'why are rockets and boosters painted white ?'],
 ['how do we prepare for upsc ?', 'how do i prepare for civil service ?']]

In [ ]:
input_size = input_data.n_words
hidden_size = 256
input_size

13735

In [ ]:
embedding = nn.Embedding(input_size, hidden_size)
gru = nn.GRU(hidden_size, hidden_size)

In [ ]:
sample = random.choice(pairs)
sample

['what exactly is communitarianism ?',
 'what is the basic concept of communitarianism ?']

In [ ]:
device

device(type='cuda')

In [ ]:
input_sentence = sample[0]
output_sentence = sample[1]

input_data.word2index['manage']

2757

In [ ]:
for word in input_sentence.split(' '):
  print(word)

what
exactly
is
communitarianism
?


In [ ]:
input_indices = [input_data.word2index[word] for word in input_sentence.split(' ')]
output_indices = [output_data.word2index[word] for word in output_sentence.split(' ')]
input_indices, output_indices

([2, 1079, 32, 4808, 9], [11, 34, 52, 2051, 1563, 58, 4742, 10])

In [ ]:
input_indices.append(EOS_Token)
output_indices.append(EOS_Token)
input_indices, output_indices

([2, 1079, 32, 4808, 9, 1], [11, 34, 52, 2051, 1563, 58, 4742, 10, 1])

In [ ]:
# send data to GPU
input_tensor = torch.tensor(input_indices, dtype=torch.long, device=device)
output_tensor = torch.tensor(output_indices, dtype=torch.long, device=device)

In [ ]:
input_tensor.shape, output_tensor.shape

(torch.Size([6]), torch.Size([9]))

In [ ]:
embedding = nn.Embedding(input_size, hidden_size).to(device)
gru = nn.GRU(hidden_size, hidden_size).to(device)

In [ ]:
embedding_input = embedding(input_tensor)
embedding_input.shape

torch.Size([6, 256])

In [ ]:
input_tensor

tensor([   2, 1079,   32, 4808,    9,    1], device='cuda:0')

In [ ]:
input_tensor.shape, input_tensor.view(-1, 1).shape

(torch.Size([6]), torch.Size([6, 1]))

In [ ]:
print(embedding_input.shape)
embedding_input = embedding(input_tensor.view(-1, 1))
print(embedding_input.shape)

torch.Size([6, 256])
torch.Size([6, 1, 256])


In [ ]:
hidden = torch.zeros(1, 1, 256, device = device)

In [ ]:
embedding_input = embedding(input_tensor.view(-1, 1))
output, hidden = gru(embedding_input, hidden)

output.shape, output[0, 0].shape

(torch.Size([6, 1, 256]), torch.Size([256]))

In [ ]:
encoder_outputs = torch.zeros(MAX_LENGTH, 256, device=device)
encoder_outputs.shape

torch.Size([10, 256])

In [ ]:
encoder_outputs = torch.zeros(MAX_LENGTH, 256, device=device)
hidden = torch.zeros(1, 1, 256, device = device)

for i in range(input_tensor.size()[0]):
  embedding_input = embedding(input_tensor[i].view(-1, 1))
  output, hidden = gru(embedding_input, hidden)
  encoder_outputs[i] += output[0, 0]

In [ ]:
encoder_outputs.shape, hidden.shape

(torch.Size([10, 256]), torch.Size([1, 1, 256]))

In [ ]:
encoder_outputs[0:8]

tensor([[ 0.3946, -0.0844, -0.0983,  ..., -0.0087, -0.0935, -0.1808],
        [-0.1303, -0.4105,  0.2691,  ..., -0.1180, -0.1057, -0.2739],
        [-0.3692, -0.3870,  0.1893,  ...,  0.2239, -0.0754, -0.3116],
        ...,
        [ 0.1222, -0.2269, -0.1849,  ..., -0.1431,  0.1225,  0.1651],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<SliceBackward0>)

# Question Similarties - Encoder Decoder architecture with Attention

![image](https://miro.medium.com/max/1838/1*tXchCn0hBSUau3WO0ViD7w.jpeg)



# Encoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Decoder with Attention

In [ ]:
decoder_input = torch.tensor([[SOS_Token]], device=device)
decoder_hidden = hidden
decoded_words = []

In [ ]:
# decoder s0
output_size = output_data.n_words
embedding = nn.Embedding(output_size, 256).to(device)
embedded = embedding(decoder_input)
embedded.shape

torch.Size([1, 1, 256])

In [ ]:
# 256 * 2 >> after concatenation
attn_weight_layer = nn.Linear(256 * 2, 10).to(device)

In [ ]:
embedded.shape, decoder_hidden.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [ ]:
torch.cat((embedded[0], decoder_hidden[0]), 1).shape

torch.Size([1, 512])

In [ ]:
attn_weight_layer = nn.Linear(256 * 2, 10).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights

tensor([[-0.1447, -0.1612, -1.0591, -0.2147,  0.9430, -0.0112, -0.4699,  0.4955,
          0.2233, -0.2299]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
import torch.nn.functional as F
attn_weight_layer = nn.Linear(256 * 2, 10).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_weights

tensor([[0.0662, 0.0897, 0.0814, 0.0831, 0.0981, 0.1334, 0.0813, 0.1120, 0.1069,
         0.1479]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [ ]:
attn_weights.shape, encoder_outputs.shape

(torch.Size([1, 10]), torch.Size([10, 256]))

In [ ]:
attn_applied = torch.bmm(attn_weights, encoder_outputs)

RuntimeError: ignored

In [ ]:
attn_weights.unsqueeze(0).shape, encoder_outputs.unsqueeze(0).shape

(torch.Size([1, 1, 10]), torch.Size([1, 10, 256]))

In [ ]:
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
attn_applied.shape

torch.Size([1, 1, 256])

In [ ]:
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
embedded.shape, attn_applied.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [ ]:
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru.shape

torch.Size([1, 256])

In [ ]:
gru = nn.GRU(256, 256).to(device)
decoder_hidden.shape, input_to_gru.shape

(torch.Size([1, 1, 256]), torch.Size([1, 256]))

In [ ]:
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
input_to_gru = input_to_gru.unsqueeze(0)
decoder_hidden.shape, input_to_gru.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [ ]:
output, decoder_hidden = gru(decoder_hidden, input_to_gru)
output.shape, decoder_hidden.shape

(torch.Size([1, 1, 256]), torch.Size([1, 1, 256]))

In [ ]:
output_word_layer = nn.Linear(256, output_data.n_words).to(device)

In [ ]:
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim=1)
output.shape, output, output.data.topk(1)

(torch.Size([1, 13757]),
 tensor([[8.4865e-05, 7.5577e-05, 6.7912e-05,  ..., 6.9525e-05, 6.9065e-05,
          6.3658e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 torch.return_types.topk(values=tensor([[0.0001]], device='cuda:0'), indices=tensor([[12508]], device='cuda:0')))

In [ ]:
topv, topi = output.data.topk(1)
output_data.index2word[topi.item()]

'handcraft'

In [ ]:
decoder_input = torch.tensor([[SOS_Token]], device=device)
decoder_hidden = hidden #decoder_hidden = encoder_hidden
output_size = output_data.n_words
embedding = nn.Embedding(output_size, 256).to(device)
embedded = embedding(decoder_input)
attn_weight_layer = nn.Linear(256 * 2, 10).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
gru = nn.GRU(256, 256).to(device)
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output_word_layer = nn.Linear(256, output_data.n_words).to(device)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_data.index2word[top_index.item()]

'ny'

In [ ]:
decoder_input = torch.tensor([[SOS_Token]], device=device)
decoder_hidden = hidden #decoder_hidden = encoder_hidden
output_size = output_data.n_words
embedding = nn.Embedding(output_size, 256).to(device)
embedded = embedding(decoder_input)
attn_weight_layer = nn.Linear(256 * 2, 10).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
gru = nn.GRU(256, 256).to(device)
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output_word_layer = nn.Linear(256, output_data.n_words).to(device)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_data.index2word[top_index.item()]

'allow'

In [ ]:
decoder_input = torch.tensor([[SOS_Token]], device=device)
decoder_hidden = hidden #decoder_hidden = encoder_hidden
output_size = output_data.n_words
embedding = nn.Embedding(output_size, 256).to(device)
embedded = embedding(decoder_input)
attn_weight_layer = nn.Linear(256 * 2, 10).to(device)
attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
attn_weights = F.softmax(attn_weights, dim = 1)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
input_to_gru_layer = nn.Linear(256 * 2, 256).to(device)
input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
gru = nn.GRU(256, 256).to(device)
input_to_gru = input_to_gru.unsqueeze(0)
output, decoder_hidden = gru(input_to_gru, decoder_hidden)
output_word_layer = nn.Linear(256, output_data.n_words).to(device)
output = F.relu(output)
output = F.softmax(output_word_layer(output[0]), dim = 1)
top_value, top_index = output.data.topk(1)
output_data.index2word[top_index.item()]

'kurt'

In [ ]:
for i in range(6):
  decoder_input = torch.tensor([[output_indices[i]]], device=device)
  decoder_hidden = hidden
  output_size = output_data.n_words
  embedded = embedding(decoder_input)
  attn_weights = attn_weight_layer(torch.cat((embedded[0], decoder_hidden[0]), 1))
  attn_weights = F.softmax(attn_weights, dim = 1)
  attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
  input_to_gru = input_to_gru_layer(torch.cat((embedded[0], attn_applied[0]), 1))
  input_to_gru = input_to_gru.unsqueeze(0)
  output, decoder_hidden = gru(input_to_gru, decoder_hidden)
  output = F.relu(output)
  output = F.softmax(output_word_layer(output[0]), dim = 1)
  top_value, top_index = output.data.topk(1)
  print(output_sentence.split(" ")[i], output_indices[i], output_data.index2word[top_index.item()], top_index.item() )
  print(attn_weights)

what 11 codeigniter 13503
tensor([[0.1092, 0.2309, 0.0257, 0.0581, 0.0860, 0.1104, 0.1086, 0.0323, 0.1318,
         0.1069]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
is 34 codeigniter 13503
tensor([[0.1253, 0.0877, 0.0513, 0.0887, 0.1302, 0.0985, 0.1979, 0.0720, 0.0812,
         0.0670]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
the 52 styles 12672
tensor([[0.0557, 0.0363, 0.1692, 0.0528, 0.1071, 0.1585, 0.1245, 0.0879, 0.0841,
         0.1239]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
basic 2051 urges 7369
tensor([[0.0489, 0.0506, 0.1148, 0.1197, 0.0909, 0.1225, 0.1254, 0.0972, 0.0906,
         0.1394]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
concept 1563 shares 324
tensor([[0.0783, 0.0705, 0.0942, 0.0557, 0.0889, 0.1179, 0.1725, 0.1266, 0.0788,
         0.1166]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
of 58 toxic 2622
tensor([[0.0758, 0.1023, 0.1166, 0.0424, 0.0723, 0.1275, 0.2467, 0.0833, 0.0689,
         0.0643]], device='cuda:0', grad_fn=<SoftmaxBackward0

In [ ]:
%matplotlib inline

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output, hidden = self.gru(output, hidden)
        output = F.relu(output)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_Token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_data, pair[0])
    target_tensor = tensorFromSentence(output_data, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_Token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_Token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))



In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_data, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_Token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_Token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_data.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_data.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_data.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

3m 53s (- 54m 24s) (5000 6%) 4.4552
7m 36s (- 49m 30s) (10000 13%) 4.0118
11m 21s (- 45m 26s) (15000 20%) 3.8560
15m 4s (- 41m 26s) (20000 26%) 3.7352
18m 48s (- 37m 37s) (25000 33%) 3.6283
22m 32s (- 33m 48s) (30000 40%) 3.5942
26m 15s (- 30m 0s) (35000 46%) 3.5702
30m 0s (- 26m 15s) (40000 53%) 3.5490
33m 42s (- 22m 28s) (45000 60%) 3.5677
37m 23s (- 18m 41s) (50000 66%) 3.5089
41m 4s (- 14m 56s) (55000 73%) 3.5384
44m 43s (- 11m 10s) (60000 80%) 3.5375
48m 20s (- 7m 26s) (65000 86%) 3.6216
51m 57s (- 3m 42s) (70000 93%) 3.7027
55m 32s (- 0m 0s) (75000 100%) 3.9088


In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> what does the number of protons determine ?
= how can you determine the number of protons ?
< what is the ? <EOS>

> does my boss secretly like me ?
=  my boss took me out for dinner
< what do i get ? <EOS>

> what is best incest movie ?
= what are movies with incest scenes ?
< what are your ? <EOS>

> who will win in punjab ?
= who will win upcoming elections in punjab ?
< how does i make ? <EOS>

> how do i find a work ?
= how can i find work ?
< how do i make ? <EOS>

>  who is scarier to be elected as president
= donald trump or hillary clinton ? 
< what s ? <EOS>

> how do silencers work on a remington rifle ?
= how do suppressors work on a remington riffle ?
< how do i make ? ? ? ? ? ?

>  what is the difference between c compiler
= c compiler and pascal compiler ? 
< what ? <EOS>

> how can i be confident without being arrogant ?
= how can i become confident but not arrogant ?
< how do i make my ? ? ? ? <EOS>

> how does it feel being raped ?
= have you ever been raped ?
< how